In [ ]:
import numpy as np
import sys
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append('../../')

from script.Model_run import Speech_analysis

In [ ]:
#Load the models
speech_analyzer = Speech_analysis(print_on = False)

In [ ]:
neurobooth_bars = pd.read_csv('/home/kvattis/Desktop/neurobooth_bars_data.csv',encoding='latin1')

In [ ]:
neurobooth_subjects = pd.read_csv('/home/kvattis/Desktop/neurobooth_bars_data_2.csv',encoding='latin1')

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
neurobooth_bars

In [ ]:
subjects = pd.merge(neurobooth_subjects,neurobooth_bars,on='subject_id', how = 'right')

In [ ]:
subjects['bars_total'] = (subjects['bars_speech']  + subjects['bars_oculomotor'] + subjects['bars_heel_shin_right'] + subjects['bars_heel_shin_left'] +  subjects['bars_finger_nose_right'] + subjects['bars_finger_nose_left'] + subjects['bars_gait'])/10. 

In [ ]:
data_file_list = glob.glob('/home/kvattis/Desktop/Neurobooth_data/*')

In [ ]:
output = pd.DataFrame(columns = ['subject_id','Neuropheno_ID','Bars_Speech','Bars_Total','Samples_no','Prob_AT_tf_grad', 'Bars_Speech_pred_t', 'Bars_Total_pred_t', 'Bars_Speech_pred_tf','Bars_Total_pred_tf'])

In [ ]:
data_file_list

In [ ]:
for dd in data_file_list:
    subject_id = int(dd[38:44])
    neuropheno_id = subjects[subjects['subject_id'] == subject_id]['neuropheno_id'].iat[0]
    bars_speech = subjects[subjects['subject_id'] == subject_id]['bars_speech'].iat[0]/10.
    bars = subjects[subjects['subject_id'] == subject_id]['bars_total'].iat[0]
    if not pd.isna(neuropheno_id):
        speech_analyzer.load(dd,neuropheno_id)
        sample_size = speech_analyzer.sample_size_s 
    else:
        speech_analyzer.load(dd,subject_id)
        sample_size = speech_analyzer.sample_size_s 
        
    #print(sample_size)
    if sample_size < 7:
        continue
    output = output.append({'subject_id' : subject_id, 'Neuropheno_ID' : neuropheno_id, 'Bars_Speech':bars_speech,'Bars_Total':bars, 'Samples_no' : speech_analyzer.sample_size_s, 'Prob_AT_tf_grad': speech_analyzer.classify(),
                               'Bars_Speech_pred_t': speech_analyzer.BARS_speech(), 'Bars_Total_pred_t': speech_analyzer.BARS_total(), 'Bars_Speech_pred_tf': speech_analyzer.BARS_speech_both_grad(), 'Bars_Total_pred_tf': speech_analyzer.BARS_total_both_grad()}, ignore_index = True)   

In [ ]:
output

In [ ]:
output__ = output.astype(float).groupby(['subject_id'], as_index = False).median().sort_values(by=['subject_id'])

In [ ]:
output_ = output__[~output__['Bars_Speech'].isna()]

In [ ]:
output_

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(output_['Bars_Speech'], output_['Prob_AT_tf_grad'], c = 'red' , marker = 'o')

plt.xlim([-0.2, 4])
plt.ylim([-0.05, 1])

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.plot([0, 4], [0, 4],color = 'k',linewidth = 4,linestyle ='-.')

ax.scatter(output_['Bars_Speech'], output_['Bars_Speech_pred_t'], c = 'red', marker = 'o', s = 60)

ax.minorticks_on()
ax.tick_params('both', length=10, width=2, which='major',direction="in")
ax.tick_params('both', length=5, width=1, which='minor',direction="in")
ax.tick_params(axis='x', labelsize=35)
ax.tick_params(axis='y', labelsize=35)
ax.set_aspect(1)
plt.xlim([-0.15, 4])
plt.ylim([-0.3, 4])
plt.xlabel(r'$BARS^{clin}_{speech}$', fontsize=40)
plt.ylabel(r'$BARS^{pred}_{speech}$', fontsize=40)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1, 1, 1)
ax.plot([0, 30], [0, 30],color = 'k',linewidth = 4,linestyle ='-.')

ax.scatter(output_['Bars_Total'], output_['Bars_Total_pred_t'],  c = 'red', marker = 'o', s = 60)

ax.minorticks_on()
ax.tick_params('both', length=10, width=2, which='major',direction="in")
ax.tick_params('both', length=5, width=1, which='minor',direction="in")
ax.tick_params(axis='x', labelsize=35)
ax.tick_params(axis='y', labelsize=35)
ax.set_aspect(1)
plt.xlim([-0.2, 30])
plt.ylim([-0.05, 30])
plt.xlabel(r'$BARS^{clin}_{Total}$', fontsize=40)
plt.ylabel(r'$BARS^{pred}_{Total}$', fontsize=40)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, mean_absolute_error

In [ ]:
mean_absolute_error(output_['Bars_Speech'].to_numpy(), output_['Bars_Speech_pred_t'].to_numpy())

In [ ]:
speech_analyzer.play_original_audio()

In [ ]:
speech_analyzer.play_resampled_audio()

In [ ]:
speech_analyzer.plot_mel()

In [ ]:
speech_analyzer.plot_mel_resampled()